In [1]:
import csv
import json
import os
import numpy as np
import nibabel as nib
import pandas as pd
import suspect

# Make a DataFrame with .rda headers' data

In [2]:
from suspect import MRSData, transformation_matrix

import numpy
import struct
import re

# The RDA format consists of a large number of key value pairs followed by raw
# data. The values need to be cast into different datatypes depending on the
# key, this dictionary stores a mapping of key to datatype.

rda_types = {
    "floats": ["PatientWeight", "TR", "TE", "TM", "TI", "DwellTime", "NumberOfAverages",
               "MRFrequency", "MagneticFieldStrength", "FlipAngle", "SliceThickness",
               "FoVHeight", "FoVWidth", "PercentOfRectFoV", "PixelSpacingRow",
               "PixelSpacingCol", "VOIPositionSag", "VOIPositionCor",
               "VOIPositionTra", "VOIThickness", "VOIPhaseFOV", "VOIReadoutFOV",
               "VOIReadoutVOV", "VOINormalSag", "VOINormalCor", "VOINormalTra",
               "VOIRotationInPlane", "FoV3D", "PixelSpacing3D"],
    "integers": ["SeriesNumber", "InstanceNumber", "AcquisitionNumber", "NumOfPhaseEncodingSteps",
                 "NumberOfRows", "NumberOfColumns", "VectorSize", "EchoNumber",
                 "NumberOf3DParts", "HammingFilterWidth", "NumberOfEchoes"],
    "strings": ["PatientID", "PatientName", "StudyDescription", "PatientBirthDate",
                "StudyDate", "StudyTime", "PatientAge", "SeriesDate", "SeriesTime",
                "SeriesDescription", "ProtocolName", "PatientPosition", "ModelName",
                "StationName", "InstitutionName", "DeviceSerialNumber", "InstanceDate",
                "InstanceTime", "InstanceComments", "SequenceName", "SequenceDescription",
                "Nucleus", "TransmitCoil", "PatientSex", "HammingFilter", "FrequencyCorrection"],
    "float_arrays": ["PositionVector", "RowVector", "ColumnVector"],
    "integer_arrays": ["CSIMatrixSize", "CSIMatrixSizeOfScan", "CSIGridShift"],
    "string_arrays": ["SoftwareVersion"],
    "dictionaries": ["TransmitRefAmplitude"]
}


def load_rda(json_el):
    global rda_types
    header_dict = {}
    for i in json_el:
        key = i
        value = json_el[i]
        if key in rda_types["strings"]:
            header_dict[key] = value
        elif key in rda_types["integers"]:
            header_dict[key] = int(value)
        elif key in rda_types["floats"]:
            header_dict[key] = float(value)
        elif "[" in key and "]" in key:
            # could be a dict or a list
            key, index = re.split("\]|\[", key)[0:2]
            if key in rda_types["dictionaries"]:
                if key not in header_dict:
                    header_dict[key] = {}
                header_dict[key][index] = value
            else:
                # not a dictionary, must be a list
                if key in rda_types["float_arrays"]:
                    value = float(value)
                elif key in rda_types["integer_arrays"]:
                    value = int(value)
                index = int(index)
                # make sure there is a list in the header_dict, with enough entries
                if not key in header_dict:
                    header_dict[key] = []
                while len(header_dict[key]) <= index:
                    header_dict[key].append(0)
                header_dict[key][index] = value
        # now we can read the data
        complex_data = []


    # some .rda files have a misnamed field, correct this here
    if "VOIReadoutFOV" not in header_dict:
        if "VOIReadoutVOV" in header_dict:
            header_dict["VOIReadoutFOV"] = header_dict.pop("VOIReadoutVOV")

    # combine positional elements in the header
    voi_size = (header_dict["VOIReadoutFOV"],
                header_dict["VOIPhaseFOV"],
                header_dict["VOIThickness"])
    voi_center = (header_dict["VOIPositionSag"],
                  header_dict["VOIPositionCor"],
                  header_dict["VOIPositionTra"])
    voxel_size = (header_dict["PixelSpacingCol"],
                  header_dict["PixelSpacingRow"],
                  header_dict["PixelSpacing3D"])

    x_vector = numpy.array(header_dict["RowVector"])
    y_vector = numpy.array(header_dict["ColumnVector"])

    to_scanner = transformation_matrix(x_vector, y_vector, numpy.array(voi_center), voxel_size)

    # put useful components from the header in the metadata
    metadata = {
        "voi_size": voi_size,
        "position": voi_center,
        "voxel_size": voxel_size,
        "protocol": header_dict["ProtocolName"],
        "to_scanner": to_scanner,
        "from_scanner": numpy.linalg.inv(to_scanner)
    }

    return MRSData(complex_data,
                   header_dict["DwellTime"] * 1e-6,
                   header_dict["MRFrequency"],
                   te=header_dict["TE"],
                   transform=to_scanner,
                   metadata=metadata)


In [34]:
csv_path = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/rda_headers.csv'
csv_path = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/rda_headers2.csv'

In [35]:
import pandas as pd

df = pd.read_csv(csv_path, sep=',', header=0).drop(columns='Unnamed: 0')
jsn = json.loads(df.to_json(orient="records"))

In [36]:
%%bash
#jupyter nbextension enable --py --sys-prefix qgridimport qgrid# only required if you have not enabled the ipywidgets nbextension yet
#jupyter nbextension enable --py --sys-prefix widgetsnbextension#to show a df simply use the below:

In [37]:
import qgrid
qgrid.show_grid(df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [38]:
df[df.PatientID=='NeuroMet031'][['PatientID', 'VOIReadoutFOV', 'VOIPhaseFOV', 'VOIThickness', 'RowVector[0]', 'RowVector[1]', 'RowVector[2]', 'ColumnVector[0]', 'ColumnVector[1]', 'ColumnVector[2]', 'VOIPositionSag', 'VOIPositionCor', 'VOIPositionTra']]

,PatientID,VOIReadoutFOV,VOIPhaseFOV,VOIThickness,RowVector[0],RowVector[1],RowVector[2],ColumnVector[0],ColumnVector[1],ColumnVector[2],VOIPositionSag,VOIPositionCor,VOIPositionTra
0,NeuroMet031,20.0,20.0,20.0,-1.0,0.0,0.0,0.0,0.898794,0.438371,-6.3,11.8,3.2
1,NeuroMet031,20.0,20.0,20.0,-1.0,0.0,0.0,0.0,0.898794,0.438371,-6.3,11.8,3.2
2,NeuroMet031,20.0,20.0,20.0,-1.0,0.0,0.0,0.0,0.898794,0.438371,-6.3,11.8,3.2


In [39]:
df2 = df.iloc[[0]]#[[148, 94, 210, 187, 121, 142, 109, 115, 289, 342]]

In [40]:
df2[['PatientID', 'RowVector[0]', 'RowVector[1]', 'RowVector[2]', 'ColumnVector[0]', 'ColumnVector[1]', 'ColumnVector[2]', 'VOIPositionSag', 'VOIPositionCor', 'VOIPositionTra']]

,PatientID,RowVector[0],RowVector[1],RowVector[2],ColumnVector[0],ColumnVector[1],ColumnVector[2],VOIPositionSag,VOIPositionCor,VOIPositionTra
0,NeuroMet031,-1.0,0.0,0.0,0.0,0.898794,0.438371,-6.3,11.8,3.2


In [41]:
df=df2

# 2 Make a Nifty with the MRS Voxel as 10mm radius sphere

In [42]:
# Credits NLTools: https://github.com/cosanlab/nltools
def sphere(nii_path, r, p):
    """ create a sphere of given radius at some point p in the brain mask
    Args:
        r: radius of the sphere
        p: point (in coordinates of the brain mask) of the center of the sphere
    """
    nii = nib.load(nii_path)
    dims = nii.shape

    x, y, z = np.ogrid[-p[0]:dims[0]-p[0], -p[1]:dims[1]-p[1], -p[2]:dims[2]-p[2]]
    mask = x*x + y*y + z*z <= r*r

    activation = np.zeros(dims)
    #print(activation.sum())
    activation[mask] = 1
    #print(activation.sum())
    #activation = np.multiply(activation, nii.get_fdata())
    activation = activation + nii.get_fdata()
    #print(activation.sum())
    activation = nib.Nifti1Image(activation, nii.affine, nii.header)

    # return the 3D numpy matrix of zeros containing the sphere as a region of ones
    return activation

In [43]:
sphere_radius = 10

In [44]:
t1_file_mask = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/Structural_analysis/{sub_id}/{sub_id}.mUNIbrain_DENskull_SPMmasked.nii.gz'

In [45]:
subject_space_sphere_path = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/Subjekt_space_spheres'
subject_space_voxel_path = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/Subject_space_Voxels'

In [46]:
ids = set([i for i in df['PatientID']])
print(ids)

{'NeuroMet031'}


In [47]:
for j in range(len(df)):
    #try:
    i=json.loads(df.iloc[j].to_json())
    print(i)
    print(type(i))
    if i['PatientID'] in ids:     
        #print(n)
        sub_id = i['PatientID']
        print(sub_id)
        t1_path = t1_file_mask.format(sub_id=sub_id)
        #print(t1_path)
        rda = load_rda(i)
        base_name = t1_path.split('/')[-1][:-7] 
        #print(base_name)
        # get voxel coordinates for the center of mrs voxel
        t1 = suspect.image.load_nifti(t1_path)
        mrs_centre = rda.to_scanner(0, 0, 0)
        #print(mrs_centre)
        mrs_centre_index = t1.from_scanner(*mrs_centre).round().astype(int)
        #print(mrs_centre_index)
        # zero all voxels and put 1 for mrs voxel center
        img = nib.load(t1_path)
        new_data = img.get_fdata().copy()
        new_data[:,:,:] = 0
        new_data[mrs_centre_index[0],mrs_centre_index[1],mrs_centre_index[2]] = 1
        #create and save nifti
        new_img = nib.Nifti1Image(new_data, img.affine, img.header)
        nib.save(new_img, os.path.join(subject_space_voxel_path, base_name + '_mrs_voxel_center.nii.gz'))
        #make a sphere and save it
        sssphere = sphere(os.path.join(subject_space_voxel_path, base_name + '_mrs_voxel_center.nii.gz'),
                     r=sphere_radius,
                     p=mrs_centre_index)
        nib.save(sssphere, os.path.join(subject_space_sphere_path, base_name + '_mrs_voxel_sphere.nii.gz'))
        #except:
    #    pass

{'PatientName': 'MDC-0131_NeuroMet031', 'PatientID': 'NeuroMet031', 'PatientSex': 'F', 'PatientBirthDate': 19410408, 'StudyDate': 20170405, 'StudyTime': 104229.75, 'StudyDescription': 'PTB^Ariane', 'PatientAge': '075Y', 'PatientWeight': 50.0, 'SeriesDate': 20170405, 'SeriesTime': 112719.421, 'SeriesDescription': 'rm_special_withOVS_32Ch_Coil', 'ProtocolName': 'rm_special_withOVS_32Ch_Coil', 'PatientPosition': 'HFS', 'SeriesNumber': 20, 'InstitutionName': 'B.U.F.F.', 'StationName': 'MRC18923', 'ModelName': 'Investigational_Device_7T', 'DeviceSerialNumber': 18923, 'SoftwareVersion[0]': 'syngo MR B17', 'InstanceDate': 20170405, 'InstanceTime': 112719.421, 'InstanceNumber': 1, 'InstanceComments': '_c_32', 'AcquisitionNumber': 1, 'SequenceName': 'SPECIAL', 'SequenceDescription': 'SPECIAL', 'TR': 6500.0, 'TE': 9.0, 'TM': 0.0, 'TI': 0.0, 'DwellTime': 250, 'EchoNumber': 0, 'NumberOfAverages': 64.0, 'MRFrequency': 297.136805, 'Nucleus': '1H', 'MagneticFieldStrength': 6.98, 'NumOfPhaseEncodingSt

In [55]:
# Coregister in MNI space

In [56]:
import ants
import os

In [57]:
mni_std = '/usr/share/fsl/data/standard/MNI152_T1_0.5mm.nii.gz'
ants_mni_std = ants.image_read(mni_std)

In [58]:
mov_mask = str(os.path.join(subject_space_sphere_path, '{sub_id}.mUNIbrain_DENskull_SPMmasked_mrs_voxel_sphere.nii.gz'))

In [59]:
transform1_mask = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MNI_coreg/{sub_id}/{sub_id}.mUNIbrain_DENskull_SPMmasked_MNIcoreg0GenericAffine.mat'
transform2_mask = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MNI_coreg/{sub_id}/{sub_id}.mUNIbrain_DENskull_SPMmasked_MNIcoreg1Warp.nii.gz'

In [60]:
ids = set([i for i in df['PatientID']])

In [61]:
for sub_id in ids:
    try:
        print(sub_id)
        transform1 = transform1_mask.format(sub_id=sub_id)
        transform2 = transform2_mask.format(sub_id=sub_id)
        moving = mov_mask.format(sub_id=sub_id)
        ants_mov = ants.image_read(moving)
        res = ants.apply_transforms(fixed=ants_mni_std,
                         moving=ants_mov,
                         transformlist = [transform2, transform1])
        ants.image_write(res, '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/MNI_space_spheres/{}_MRS_Voxel_MNIwarped.nii.gz'.format(sub_id))
    except:
        pass

NeuroMet031
